Датасет:
https://www.kaggle.com/c/kkbox-music-recommendation-challenge/overview

Можете мерять результат по 5-fold evaluation на train-set.

Soft dead:
25.10
Dead dead:
01.11

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp '/content/drive/My Drive/AU/kaggle.json' ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c kkbox-music-recommendation-challenge

 60% 25.0M/41.9M [00:00<00:00, 29.5MB/s]
100% 41.9M/41.9M [00:00<00:00, 57.4MB/s]
  0% 0.00/453k [00:00<?, ?B/s]
100% 453k/453k [00:00<00:00, 143MB/s]
 96% 97.0M/101M [00:01<00:00, 50.0MB/s]
100% 101M/101M [00:01<00:00, 69.7MB/s] 
 90% 89.0M/98.8M [00:01<00:00, 48.6MB/s]
100% 98.8M/98.8M [00:01<00:00, 78.6MB/s]
 84% 85.0M/101M [00:01<00:00, 39.1MB/s]
100% 101M/101M [00:01<00:00, 74.6MB/s] 
  0% 0.00/1.29M [00:00<?, ?B/s]
100% 1.29M/1.29M [00:00<00:00, 87.6MB/s]


# Импорты

In [ ]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MultiLabelBinarizer
import datetime
import matplotlib.pyplot as plt
from re import split
from tqdm.auto import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import lightgbm as lgb
from gensim.models import Word2Vec
from collections import defaultdict
import multiprocessing

# Данные

In [ ]:
!7z x "members.csv.7z"
!7z x "song_extra_info.csv.7z"
!7z x "songs.csv.7z"
!7z x "test.csv.7z"
!7z x "train.csv.7z"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 1349856 bytes (1319 KiB)

Extracting archive: members.csv.7z
--
Path = members.csv.7z
Type = 7z
Physical Size = 1349856
Headers Size = 130
Method = LZMA2:3m
Solid = -
Blocks = 1

  0%    Everything is Ok

Size:       2503827
Compressed: 1349856

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 103608205 bytes (99 MiB)

Extracting archive: song_extra_info.csv.7z
--
Path = song_extra_info.csv.7z
Type = 7z
Physical Size = 103608205
Headers Size = 140
Method = LZMA:25
Solid = -
Blocks = 1

 

In [ ]:
dataset_path = ""
members = pd.read_csv(dataset_path + "members.csv", dtype={'city': 'category',
                                                           'registered_via': 'category'},
                      parse_dates=['registration_init_time', 'expiration_date'])
song_extra_info = pd.read_csv(dataset_path + "song_extra_info.csv", dtype={'name': 'category',
                                                                           'song_id': 'category'})
songs = pd.read_csv(dataset_path + "songs.csv", dtype={'language': 'category',
                                                       'song_id': 'category'})
test = pd.read_csv(dataset_path + "test.csv", dtype={'msno': 'category',
                                                     'source_system_tab': 'category',
                                                     'source_screen_name': 'category',
                                                     'source_type': 'category',
                                                     'song_id': 'category'})
train = pd.read_csv(dataset_path + "train.csv", dtype={'msno': 'category',
                                                       'source_system_tab': 'category',
                                                       'source_screen_name': 'category',
                                                       'source_type': 'category',
                                                       'song_id': 'category',
                                                       'target': int})

In [ ]:
members

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,2011-08-20,2017-09-20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,2015-06-28,2017-06-22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,2016-04-11,2017-07-12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,2015-09-06,2015-09-07
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,2017-01-26,2017-06-13
...,...,...,...,...,...,...,...
34398,Wwd/cudKVuLJ3txRVxlg2Zaeliu+LRUfiBmfrnxhRCY=,1,0,NaN,7,2013-11-11,2017-09-10
34399,g3JGnJX6Hg50lFbrNWfsHwCUmApIkiv2M8sXOaeXoIQ=,4,18,male,3,2014-10-24,2017-05-18
34400,IMaPMJuyN+ip9Vqi+z2XuXbFAP2kbHr+EvvCNkFfj+o=,1,0,NaN,7,2013-08-02,2017-09-08
34401,WAnCAJjUty9Stv8yKtV7ZC7PN+ilOy5FX3aIJgGPANM=,1,0,NaN,7,2015-10-20,2017-09-20


In [ ]:
songs

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
...,...,...,...,...,...,...,...
2296315,lg6rn7eV/ZNg0+P+x77kHUL7GDMfoL4eMtXxncseLNA=,20192,958,Catherine Collard,Robert Schumann (1810-1856),NaN,-1.0
2296316,nXi1lrSJe+gLoTTNky7If0mNPrIyCQCLwagwR6XopGU=,273391,465,紀文惠 (Justine Chi),NaN,NaN,3.0
2296317,9KxSvIjbJyJzfEVWnkMbgR6dyn6d54ot0N5FKyKqii8=,445172,1609,Various Artists,NaN,NaN,52.0
2296318,UO8Y2MR2sjOn2q/Tp8/lzZTGKmLEvwZ20oWanG4XnYc=,172669,465,Peter Paul & Mary,NaN,NaN,52.0


In [ ]:
song_extra_info

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001
...,...,...,...
2295966,hLnetpF6UbPg28sSfXnPE2vsdaGsLvddlXEdJR4VTIA=,Deep Breathing,PLL431720793
2295967,N+6vJ8actKQm0S3Fpf4elipTjoAo9ev28aA5FJN5e40=,In Hiding,US5UL1519827
2295968,pv35uG0ts05mWtirM/AMOWEzbHxIVart5ZzRXqKUY1c=,Il Est Ne Le Divin Enfant,PLL431502294
2295969,QSySnm8jt2Go7byY34/PxsZP6dPCins2j2cyYquNhBo=,The Exodus Song,DEPZ69316095


In [ ]:
train

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1
...,...,...,...,...,...,...
7377413,6xdFzPlrasIDD95mQWXVC3Bg4ptnGYtBl4ztVEZMddU=,VJTxizih/o28kXCbtPbIyWXScoXGvxyYtl6R+0YB5JM=,my library,Local playlist more,local-playlist,1
7377414,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,z1mqaU9YOX7T/PFDvUoWozdFq7rC3KwaQP7nFVprjMI=,search,Search,song,0
7377415,ZxbVmt3Kh/XOH+h58c2Kdj6SjFZk+wnUO006IgWzMQE=,750RprmFfLV0bymtDH88g24pLZGVi5VpBAI300P6UOA=,search,Search,song,1
7377416,0aH4Hd3ziPSRHClRX8rkeOEaAG5EPPkW1mKGCdXEok0=,G8wgqObgeAMER/rVCIlgcNeQ8mm0CzF/GsxiMK8TTnA=,discover,Discover Genre,online-playlist,1


Сделаем функцию, которая из датасета будет доставать фичи.

In [ ]:
class FeatureExtractor:
    def __init__(self, members, songs, song_extra_info):
        self.members = members.copy()
        self.songs = songs.copy()
        self.song_extra_info = song_extra_info.copy()
        self.prepare_members()
        self.prepare_songs()
        
    def prepare_members(self):
        print('Prepare members...')
        # Убираю аутлайеры из возраста и завожу отдельную фичу для людей, которые не указали возраст
        bd = self.members.loc[:, 'bd'].copy().astype(int)
        bd_missing = (bd <= 0) | (bd > 100)
        bd.loc[bd_missing] = 0
        self.members.loc[:, 'bd'] = bd.astype(int)
        self.members['bd_missing'] = bd_missing.astype(int)
        
        # Преобразую время
        print('Prepare dates...')
        dates = ['registration_init_time', 'expiration_date']
        for d in dates:
            self.members[d +'_year'] = self.members[d].dt.year
            self.members[d +'_month'] = self.members[d].dt.month
            self.members[d +'_day'] = self.members[d].dt.day
        self.members.drop(columns=dates, inplace=True)
        
        # Убираю NaN'ы у пола
        print('Prepare genders...')
        self.members['gender'] = self.members['gender'].fillna('miss').astype('category')
        
    def prepare_songs(self):
        print('Prepare songs...')
        # Делаю one-hot фичи
        # Попытался сделать ван хот, но не влезло в память :(
        # one_hot_classes = ['genre_ids']
        # to_one_hot = songs[one_hot_classes].copy()
        # to_one_hot.fillna('miss', inplace=True)
        # print('Songs to one-hot...')
        # for cl in tqdm(one_hot_classes):
        #     songs_one_hot_encoder = MultiLabelBinarizer()
        #     one_hot = songs_one_hot_encoder.fit_transform(to_one_hot[cl].apply(FeatureExtractor.split_string))
        #     categories = [cl + '_' + str(s) for s in songs_one_hot_encoder.classes_]
        #     df = pd.DataFrame(one_hot, columns=categories)
        #     self.songs = self.songs.merge(df, left_index=True, right_index=True, validate='1:1')
        classes_to_delete = ['composer', 'lyricist']
        self.songs.drop(columns=classes_to_delete, inplace = True)
        self.songs['artist_name'] = self.songs['artist_name'].astype('category')
        self.songs['genre_ids'] = self.songs['genre_ids'].astype('category')
            
        # Инфа из isrc
        print('Extract data from isrc...')
        isrc = self.song_extra_info['isrc'].transform([self.song_year, self.song_country])
        print('Merging...')
        self.song_extra_info['song_country'] = isrc.song_country.astype('category')
        self.song_extra_info['song_year'] = isrc.song_year
        self.song_extra_info.drop(columns=['name', 'isrc'], inplace=True)
    
    # Функция разделения строки по |, / и т.п символам
    @staticmethod
    def split_string(string):
        return list(filter(None, map(lambda t: t.strip(), split(r'\||/|\(|\)|feat.|,', string.lower()))))
    
    # isrc to year
    def song_year(self, isrc):
        if type(isrc) == str:
            if int(isrc[5:7]) > 17:
                return 1900 + int(isrc[5:7])
            else:
                return 2000 + int(isrc[5:7])
        else:
            return np.nan
        
    # isrc to country
    def song_country(self, isrc):
        if type(isrc) == str:
            return isrc[:2]
        else:
            return 'miss'

    def get_features(self, dataset):
        return dataset.merge(self.members, on='msno', how='left').merge(self.songs, on='song_id', how='left').merge(self.song_extra_info, on='song_id', how='left')        

In [ ]:
features = FeatureExtractor(members, songs, song_extra_info)

Prepare members...
Prepare dates...
Prepare genders...
Prepare songs...
Extract data from isrc...
Merging...


In [ ]:
X_train = features.get_features(train)

In [ ]:
X_train, y_train = X_train.drop(columns=['msno', 'song_id', 'target']), X_train.target

# Задача 1:

Собрать более-менее приличную рекомендательную систему с помощью GBDT. Использование трюков из лекции (и не из неё), тюнинг и разумных объемов фича инжиниринг поощряется. 

In [ ]:
def train_lgb(X_train, X_test, y_train, y_test):
    train = lgb.Dataset(X_train, y_train)
    val = lgb.Dataset(X_test, y_test)
    params = {'learning_rate': 0.2,
              'objective': 'binary',
              'metric': 'auc',
              'max_depth': 10,
              'num_leaves': 256,
              'max_depth': 10}
    model = lgb.train(params,
                    train_set=train,
                    num_boost_round=100,
                    verbose_eval=10,
                    valid_sets=[train, val])
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)
    return model, auc

In [ ]:
def k_fold(X, y, k=5):
    scores = []
    sc = 0
    kf = StratifiedKFold(n_splits=k, shuffle=True)
    for train_idxs, test_idxs in tqdm(kf.split(X, y), total=k):
        print(f"k={len(scores)+1}")
    
        X_train, y_train = X.iloc[train_idxs], y.iloc[train_idxs]
        X_test, y_test = X.iloc[test_idxs], y.iloc[test_idxs]

        model, score = train_lgb(X_train, X_test, y_train, y_test)
        scores.append(score)
        if score > sc:
            sc = score
            best_model = model
    print(f'Mean ROC-AUC: {np.mean(scores):.3f}')
    return best_model

In [ ]:
data = train_test_split(X_train, y_train, test_size=0.1)

In [ ]:
model, auc = train_lgb(*data)
print("ROC-AUC:", auc)

/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's auc: 0.703878	valid_1's auc: 0.700093
[20]	training's auc: 0.713306	valid_1's auc: 0.707716
[30]	training's auc: 0.719563	valid_1's auc: 0.71253
[40]	training's auc: 0.724246	valid_1's auc: 0.71592
[50]	training's auc: 0.727143	valid_1's auc: 0.717787
[60]	training's auc: 0.730934	valid_1's auc: 0.720303
[70]	training's auc: 0.733549	valid_1's auc: 0.721898
[80]	training's auc: 0.736204	valid_1's auc: 0.723666
[90]	training's auc: 0.739439	valid_1's auc: 0.7258
[100]	training's auc: 0.741651	valid_1's auc: 0.72713
ROC-AUC: 0.7271303763707755


In [ ]:
bmodel = k_fold(X_train, y_train)

k=1


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's auc: 0.704678	valid_1's auc: 0.699262
[20]	training's auc: 0.713003	valid_1's auc: 0.705812
[30]	training's auc: 0.719154	valid_1's auc: 0.71042
[40]	training's auc: 0.724055	valid_1's auc: 0.713959
[50]	training's auc: 0.728221	valid_1's auc: 0.716856
[60]	training's auc: 0.731639	valid_1's auc: 0.719121
[70]	training's auc: 0.735795	valid_1's auc: 0.722076
[80]	training's auc: 0.738724	valid_1's auc: 0.72387
[90]	training's auc: 0.741964	valid_1's auc: 0.725875
[100]	training's auc: 0.744324	valid_1's auc: 0.727267
k=2


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's auc: 0.704845	valid_1's auc: 0.70057
[20]	training's auc: 0.712836	valid_1's auc: 0.706412
[30]	training's auc: 0.719607	valid_1's auc: 0.711538
[40]	training's auc: 0.723638	valid_1's auc: 0.714271
[50]	training's auc: 0.727695	valid_1's auc: 0.717067
[60]	training's auc: 0.730431	valid_1's auc: 0.718806
[70]	training's auc: 0.732971	valid_1's auc: 0.720322
[80]	training's auc: 0.736103	valid_1's auc: 0.722322
[90]	training's auc: 0.739834	valid_1's auc: 0.724825
[100]	training's auc: 0.74302	valid_1's auc: 0.726773
k=3


...

[10]	training's auc: 0.703638	valid_1's auc: 0.699906
[20]	training's auc: 0.711804	valid_1's auc: 0.706103
[30]	training's auc: 0.718173	valid_1's auc: 0.710947
[40]	training's auc: 0.722594	valid_1's auc: 0.714037
[50]	training's auc: 0.727244	valid_1's auc: 0.717306
[60]	training's auc: 0.730239	valid_1's auc: 0.719164
[70]	training's auc: 0.733029	valid_1's auc: 0.720991
[80]	training's auc: 0.735199	valid_1's auc: 0.722268
[90]	training's auc: 0.738366	valid_1's auc: 0.724281
[100]	training's auc: 0.74172	valid_1's auc: 0.72642
k=4


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's auc: 0.703529	valid_1's auc: 0.698442
[20]	training's auc: 0.712674	valid_1's auc: 0.705603
[30]	training's auc: 0.719432	valid_1's auc: 0.710732
[40]	training's auc: 0.723228	valid_1's auc: 0.713264
[50]	training's auc: 0.727511	valid_1's auc: 0.716129
[60]	training's auc: 0.730346	valid_1's auc: 0.717919
[70]	training's auc: 0.733373	valid_1's auc: 0.719841
[80]	training's auc: 0.735953	valid_1's auc: 0.721389
[90]	training's auc: 0.740302	valid_1's auc: 0.724265
[100]	training's auc: 0.742356	valid_1's auc: 0.725323
k=5


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[10]	training's auc: 0.70397	valid_1's auc: 0.699127
[20]	training's auc: 0.71248	valid_1's auc: 0.705418
[30]	training's auc: 0.718336	valid_1's auc: 0.709753
[40]	training's auc: 0.723162	valid_1's auc: 0.713192
[50]	training's auc: 0.72754	valid_1's auc: 0.716145
[60]	training's auc: 0.731783	valid_1's auc: 0.719008
[70]	training's auc: 0.734266	valid_1's auc: 0.720522
[80]	training's auc: 0.736969	valid_1's auc: 0.722211
[90]	training's auc: 0.739846	valid_1's auc: 0.723938
[100]	training's auc: 0.743817	valid_1's auc: 0.726501

Mean ROC-AUC: 0.726


# Задача 2: 
Пользуясь информацией о треках построить неклассические эмбеддинги и попробовать использовать их:

1. показать симилары глазами
2. в качестве рекоммендера вместо GBDT (разницу в метриках) 
3. стек с GBDT (прирост / отсутствие / почему (советую поизучать SHAP values)

In [151]:
!pip install -q swifter
import swifter

     |████████████████████████████████| 634kB 5.3MB/s 
     |████████████████████████████████| 471kB 24.5MB/s 
     |████████████████████████████████| 542kB 31.1MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 22.9MB 1.7MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
     |████████████████████████████████| 2.9MB 49.5MB/s 
     |████████████████████████████████| 133kB 60.9MB/s 
     |████████████████████████████████| 71kB 10.2MB/s 
     |████████████████████████████████| 204kB 34.6MB/s 
     |████████████████████████████████| 1.3MB 47.3MB/s 
     |████████████████████████████████| 81kB 10.3MB/s 
     |████████████████████████████████| 61kB 9.5MB/s 
     |████████████████████████████████| 296kB 59.2MB/s 
     |████████████████████████████████| 143kB 61.3MB/s 
     |████████████████████████████████| 102kB 13.4MB/s 
ERROR: modin 0.8.1.1 has requirement pandas==1.1.2, but you'll have pandas 1.1.3 which is incompatible.


In [1]:
class W2V:
    def __init__(self, embedding_size=32, user_songs=None, w2v=None, user_embeddings=None):
        self.embedding_size = embedding_size
        self.user_embeddings = user_embeddings
        self.user_songs = user_songs
        self.w2v = w2v

    def build_user_embeddings(self, X_train):
        print('Building user embeddings...')
        user_ids = X_train.msno.unique().tolist()
        for user_id in tqdm(user_ids):
            if user_id in self.user_songs:
                v = [self.w2v.wv[s_id] for s_id in self.user_songs[user_id] if s_id in self.w2v.wv]
                if len(v) > 0:
                    self.user_embeddings[user_id] = np.mean(v)

    def build_sentenses(self, X_train, y_train):
        print('Building user songs...')
        for (index, row), target in tqdm(zip(X_train.iterrows(), y_train), total=y_train.size):
            if target == 1:
                user_id = row['msno']
                song_id = row['song_id']
                self.user_songs[user_id].append(song_id)

    def train(self, X_train, y_train):
        if self.user_songs is None:
            self.user_songs = defaultdict(list)
            self.build_sentenses(X_train, y_train)

        sentenses = list(self.user_songs.values())

        if self.w2v is None:
            self.w2v = Word2Vec(min_count=20,
                     window=8,
                     size=self.embedding_size,
                     negative=20,
                     sg=1,
                     workers=multiprocessing.cpu_count())
        
            print('Building vocabulary...')
            self.w2v.build_vocab(sentenses, progress_per=1000)
            
            print('Training Word2Vec...')
            self.w2v.train(sentenses, total_examples=self.w2v.corpus_count, epochs=10, report_delay=1)
            print(self.w2v)
            
            self.w2v.init_sims(replace=True)

        if self.user_embeddings is None:
            self.user_embeddings = defaultdict(lambda: np.zeros(self.embedding_size))
            self.build_user_embeddings(X_train)

    def most_similar(self, song_id):
        return self.w2v.wv.most_similar(positive=song_id)

    def predict_proba(self, X):
        song2emb = lambda song: self.w2v.wv.word_vec(song) if song in self.w2v.wv.vocab else np.zeros(self.embedding_size)
        user2emb = lambda user: self.user_embeddings[user]
        s_emb = np.vstack(X['song_id'].astype(str).apply(song2emb))
        u_emb = np.vstack(X['msno'].astype(str).apply(user2emb))
        return (s_emb * u_emb).sum(axis=1)

    def roc_auc(self, X_train, y_train):
        return roc_auc_score(self.predict_proba(X_train), y_train)

In [189]:
user_songs = word2vec.user_songs
w2v = word2vec.w2v
user_embeddings = word2vec.user_embeddings

In [2]:
word2vec = W2V(32, user_songs, w2v, user_embeddings)
word2vec.train(train.drop(columns='target'), train.target)

NameError: ignored

1) Симилары для Eminem

In [ ]:
for s_id in iter(songs[songs['artist_name'] == 'Eminem']['song_id']):
    try:
        print(s_id, word2vec.w2v.wv[s_id])
    except:
        pass

p+WxS35GIo9lHYpW17Bz/PRW+UcGH9R+bq+wU2caYnY= [ 0.17438029 -0.26580173  0.1187593  -0.04456972  0.1310327   0.14094657
  0.02276999 -0.19096397  0.10066774  0.03077202  0.06000805  0.19961043
 -0.12387656  0.19522561 -0.08482481  0.017716   -0.11607488  0.06358662
  0.16272493  0.0400308  -0.00596581  0.09035423 -0.30928543  0.19801499
 -0.2814832  -0.13761885 -0.27708662  0.25196648 -0.27834594  0.35692072
 -0.23658754  0.0734304 ]
nfhjNo8jBDgSEFM0c05FVA2vxoaYAJnD+UCgJgLcwMo= [ 0.0278043  -0.31557512 -0.16735604 -0.06408088 -0.09566201  0.1568692
  0.05995512 -0.26435268  0.09016035 -0.01910402 -0.09122843  0.01304325
 -0.08901723  0.05382768  0.00366709  0.10946743 -0.12450722  0.39101532
  0.20056428  0.17640427  0.22019623  0.16592592 -0.15589166  0.33684805
 -0.15549321 -0.20987794 -0.21644293  0.14738776 -0.10488506  0.10777333
 -0.271174    0.1844154 ]
Y6L4QRgWFt+9pMxGusbAiWXIwzGRj29hWN85F1tN+p4= [ 0.03295499 -0.07473782  0.17504145 -0.13801987 -0.19863708  0.1109778
  0.06336665

In [ ]:
song_id = 'p+WxS35GIo9lHYpW17Bz/PRW+UcGH9R+bq+wU2caYnY='
sims, sc = tuple(zip((song_id, 1), *word2vec.most_similar(song_id)))
res = songs.set_index('song_id').loc[list(sims)]
res['scores'] = sc
res

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,song_length,genre_ids,artist_name,composer,lyricist,language,scores
song_id,,,,,,,
p+WxS35GIo9lHYpW17Bz/PRW+UcGH9R+bq+wU2caYnY=,326426,1259,Eminem,Jeff Bass| Luis Resto| Marshall Mathers,NaN,52.0,1.000000
mQiuYoYa7M4AXaoBIdnsYdWsQvdxO4+iOLEJDPH2uHA=,248081,1259,Eminem,Marshall Mathers| Luis Resto| M. Samuels| J. E...,NaN,52.0,0.952463
gET0e9X1wUcrjalriOT7N3/Tlo2c+qO2WFHd5+LZs4s=,250932,1259,Eminem,Marshall Mathers| B. Fryzel| A. Kleinstub| M. ...,NaN,52.0,0.903580
nYo7Vg+L8TvlFXWOjab5MaVY/dDn0ReI0tsuGZ0cLWA=,303809,1259,Bad Meets Evil,Marshall Mathers| R. Montgomery| P. Hernandez|...,NaN,52.0,0.880711
pBwVrte2VIS1O76/z7Km699cNPbwtsoHoKoFMQZPT8Q=,179800,1259,B.o.B,NaN,NaN,52.0,0.879951
78w25Ysgx+7z9JWQwlA3ypIpPvKBz6afDGLnp1OZUcw=,250125,1259,Eminem,Marshall Mathers| B. Fryzel| A. Kleinstub| M. ...,NaN,52.0,0.876184
4L5G2OzB1D1yecg4aIA69qjZ7C4CRgsFp66ZFb7ZKQI=,235624,1259,Macklemore & Ryan Lewis,NaN,NaN,52.0,0.869271
PFGzXV3clS2Gph9GoNMmszd+SaHFJia2VC1Np/btxRM=,192208,1259,Flo Rida,NaN,NaN,52.0,0.856436
wE383tsbzVZPCJrYGJSDKwlLiHMl5im1sABLZNbZMA8=,277246,1259,Jay-Z,Alicia Keys| S. Robinson| Bobby B. Keyes| Shaw...,NaN,52.0,0.855868


Eminem есть в симиларах, значит все хорошо

2) Подсчет метрик

In [ ]:
predicted_scores = word2vec.predict_proba(train)

In [147]:
len(train.target), len(predicted_scores)

(7377418, 5820067)

In [146]:
roc_auc_score(train.target, predicted_scores)

ValueError: ignored

3) использование `predicted_scores` в качестве фичи

In [ ]:
train['predicted_scores'] = predicted_scores

In [ ]:
X_train = features.get_features(train)

In [ ]:
X_train, y_train = X_train.drop(columns=['msno', 'song_id', 'target']), X_train.target